## Libraries

In [ ]:
import os
import shutil
import tensorflow as tf
import json
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from glob import glob
from sklearn.utils import class_weight
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

## Mount Google Drive

In [ ]:
drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/TB-Classification/tbx11k_dataset'
print(os.path.exists(base_dir))

## Paths and Directories




In [ ]:
source_dir = '/content/drive/MyDrive/TB-Classification/tbx11k_dataset'
local_dir = '/content/tbx11k_dataset'

def count_files(directory):
    if not os.path.exists(directory):
        return 0
    return sum([len(files) for r, d, files in os.walk(directory)])

print(f"Checking dataset integrity...")

num_source_files = count_files(os.path.join(source_dir, 'train'))
print(f"Source (Drive) has {num_source_files} training files.")

num_local_files = count_files(os.path.join(local_dir, 'train'))
print(f"Local (Colab) has {num_local_files} training files.")

if num_local_files != num_source_files or num_source_files == 0:
    print("Mismatch or empty folder detected. Deleting and re-copying...")

    if os.path.exists(local_dir):
        shutil.rmtree(local_dir)

    print("Starting fresh copy...")
    shutil.copytree(source_dir, local_dir)
    print("Copy complete!")
else:
    print("Dataset is complete and matches source. Skipping copy.")

BASE_DIR = local_dir
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TRAIN_ANN_DIR = os.path.join(BASE_DIR, 'train', 'ann')
VAL_DIR = os.path.join(BASE_DIR, 'val')
MODELS_DIR = '/content/drive/MyDrive/TB-Classification/tbx11k_dataset/models'
os.makedirs(MODELS_DIR, exist_ok=True)
checkpoint_path = os.path.join(MODELS_DIR, 'best_tb_model.keras')

## Dataset Mapping

In [ ]:
LABEL_MAP = {
    'healthy': 'Healthy',
    'sick_but_non-tb': 'Sick_but_Non_TB',
    'active_tb': 'Active_TB',
    'latent_tb': 'Latent_TB',
    'active&latent_tb': 'Latent_TB',
    'UNKNOWN': None
}

TARGET_CLASSES = ['Healthy', 'Sick_but_Non_TB', 'Active_TB', 'Latent_TB']

def create_dataframe(directory):
    img_dir = os.path.join(directory, 'img')
    ann_dir = os.path.join(directory, 'ann')
    data = []

    if not os.path.exists(ann_dir):
        print(f"Warning: Annotation directory not found: {ann_dir}")
        return pd.DataFrame()

    json_files = glob(os.path.join(ann_dir, '*.json'))
    print(f"Scanning {len(json_files)} annotations in {directory}...")

    for j_file in json_files:
        try:
            with open(j_file, 'r') as f:
                content = json.load(f)

            raw_label = None
            if 'tags' in content and len(content['tags']) > 0:
                raw_label = content['tags'][0].get('name')
            elif 'objects' in content and len(content['objects']) > 0:
                raw_label = content['objects'][0].get('classTitle')

            if not raw_label: continue

            mapped_label = LABEL_MAP.get(raw_label)
            if mapped_label is None or mapped_label not in TARGET_CLASSES:
                continue

            base_name = os.path.basename(j_file)
            for ext in ['.json', '.png', '.jpg', '.jpeg']:
                base_name = base_name.replace(ext, '')

            img_path = None
            for ext in ['.png', '.jpg', '.jpeg']:
                temp_path = os.path.join(img_dir, base_name + ext)
                if os.path.exists(temp_path):
                    img_path = temp_path
                    break

            if img_path:
                data.append({'filename': img_path, 'class': mapped_label})

        except Exception as e:
            print(f"Error on {j_file}: {e}")
            continue

    df = pd.DataFrame(data)
    print(f"Found {len(df)} images in {directory}")
    return df

LABEL_MAP = {
    'healthy': 'Healthy',
    'sick_but_non-tb': 'Sick_but_Non_TB',
    'active_tb': 'Active_TB',
    'latent_tb': 'Latent_TB',
    'active&latent_tb': 'Latent_TB',
    'UNKNOWN': None
}
TARGET_CLASSES = ['Healthy', 'Sick_but_Non_TB', 'Active_TB', 'Latent_TB']

print("--- Loading Data ---")
train_df = create_dataframe(TRAIN_DIR)
val_df = create_dataframe(VAL_DIR)

if train_df.empty:
    raise ValueError("CRITICAL ERROR: No training data found. Check your TRAIN_DIR path.")

print(f"\n--- Class Distribution BEFORE Balancing ---\n{train_df['class'].value_counts()}")

target_n = 2000

def balanced_sample(df_subset, target_n):
    if len(df_subset) == 0:
        return df_subset
    if len(df_subset) < target_n:
        print(f"Upsampling {df_subset['class'].iloc[0]} from {len(df_subset)} to {target_n}")
        return resample(df_subset, replace=True, n_samples=target_n, random_state=42)

    else:
        print(f"Downsampling {df_subset['class'].iloc[0]} from {len(df_subset)} to {target_n}")
        return resample(df_subset, replace=False, n_samples=target_n, random_state=42)

df_healthy = train_df[train_df['class'] == 'Healthy']
df_sick = train_df[train_df['class'] == 'Sick_but_Non_TB']
df_active = train_df[train_df['class'] == 'Active_TB']
df_latent = train_df[train_df['class'] == 'Latent_TB']

train_df_balanced = pd.concat([
    balanced_sample(df_healthy, 3000),
    balanced_sample(df_sick, 3000),
    balanced_sample(df_active, target_n),
    balanced_sample(df_latent, target_n)
])

train_df = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n--- Class Distribution AFTER Balancing ---\n{train_df['class'].value_counts()}")

## Configuration and Hyperparameters

In [ ]:
CONFIG = {
    "IMG_HEIGHT": 224,
    "IMG_WIDTH": 224,
    "BATCH_SIZE": 32,
    "EPOCHS": 20,
    "LEARNING_RATE": 1e-4,
    "DROPOUT_RATE": 0.5,
    "L2_REG": 0.001
}

## Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

print("Data Generators created.")

## Generators

In [ ]:
print("Initializing Data Generators from DataFrames...")

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(CONFIG["IMG_HEIGHT"], CONFIG["IMG_WIDTH"]),
    batch_size=CONFIG["BATCH_SIZE"],
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(CONFIG["IMG_HEIGHT"], CONFIG["IMG_WIDTH"]),
    batch_size=CONFIG["BATCH_SIZE"],
    class_mode='categorical',
    shuffle=False
)

## Imbalance Handling

In [ ]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
train_class_weights = dict(enumerate(class_weights))

print(f"Class Weights: {train_class_weights}")

## Focal Loss Management

In [ ]:
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from Lin et al., 2017.
    Arguments:
        gamma (float): Focusing parameter. Higher values focus more on hard examples.
        alpha (float): Balancing parameter.
    """
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        cross_entropy = -y_true * K.log(y_pred)

        weight = alpha * y_true * K.pow((1 - y_pred), gamma)

        loss = weight * cross_entropy
        return K.sum(loss, axis=1)

    return focal_loss_fixed

tf.keras.utils.get_custom_objects().update({'focal_loss_fixed': categorical_focal_loss})

## VGG-Style CNN Architecture

In [ ]:
def build_tuned_cnn():
    regularizer = l2(CONFIG["L2_REG"])

    model = Sequential([
        Input(shape=(CONFIG["IMG_HEIGHT"], CONFIG["IMG_WIDTH"], 3)),

        Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        GlobalAveragePooling2D(),

        Dense(256, activation='relu', kernel_regularizer=regularizer),
        Dropout(CONFIG["DROPOUT_RATE"]),

        Dense(128, activation='relu', kernel_regularizer=regularizer),
        Dropout(0.3),

        Dense(4, activation='softmax', dtype='float32')
    ])
    return model

resume_checkpoint_path = os.path.join(MODELS_DIR, 'best_tb_model.keras')

if os.path.exists(resume_checkpoint_path):
    print(f"Found checkpoint at: {resume_checkpoint_path}")
    print("Resuming training...")
    model = load_model(resume_checkpoint_path, custom_objects={'focal_loss_fixed': categorical_focal_loss()})
    print("Model loaded successfully!")

else:
    print(f"⚠️ No checkpoint found. Building fresh model...")
    model = build_tuned_cnn()

    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG["LEARNING_RATE"])

    model.compile(
        optimizer=optimizer,
        loss=categorical_focal_loss(gamma=2.0, alpha=0.25),
        metrics=[
            'accuracy',
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.Precision(name='precision')
        ]
    )
    print("✅ New model compiled with Focal Loss and Recall metrics.")

model.summary()

## Training and Logging

In [ ]:
LOGS_DIR = os.path.join(BASE_DIR, 'logs')
os.makedirs(LOGS_DIR, exist_ok=True)

log_path = os.path.join(LOGS_DIR, 'training_log.csv')
best_model_path = os.path.join(MODELS_DIR, 'best_tb_model.keras')
last_model_path = os.path.join(MODELS_DIR, 'last_tb_model.keras')

print(f"📂 Logs directory: {LOGS_DIR}")
print(f"💾 Best Model path: {best_model_path}")
print(f"💾 Last Model path: {last_model_path}")


# --- STEP 1: Determine Start Epoch ---

initial_epoch = 0

if os.path.exists(log_path):
    try:
        log_df = pd.read_csv(log_path)
        if not log_df.empty:
            # The CSV logs epochs starting from 0.
            # If the max epoch in CSV is 5, we have finished epoch 5.
            # So we should start at 6.
            last_recorded_epoch = log_df['epoch'].max()
            initial_epoch = last_recorded_epoch + 1
            print(f"📄 Log file found. Resuming from Epoch {initial_epoch} (0-indexed).")
        else:
            print("📄 Log file is empty. Starting from Epoch 0.")
    except Exception as e:
        print(f"⚠️ Error reading log file: {e}. Starting from Epoch 0.")
else:
    print("📄 No log file found. Starting from Epoch 0.")


# --- STEP 2: Define Callbacks ---

# 1. Logging Callback (Append mode is crucial for resuming)
csv_logger = CSVLogger(log_path, append=True)

# 2. Checkpointing - BEST Model
# This saves the model only when validation accuracy improves.
checkpoint_best = ModelCheckpoint(
    best_model_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# 3. Checkpointing - LAST Model (Crucial for Resuming)
# This saves the model at the end of *every* epoch.
# If training crashes, this file represents the most recent state.
checkpoint_last = ModelCheckpoint(
    last_model_path,
    save_best_only=False, # Save regardless of performance
    verbose=0
)

# 4. Dynamic Learning Rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,       # Changed from 0.5 to 0.2 (more aggressive drop)
    patience=3,       # Increased patience slightly
    min_lr=1e-7,
    verbose=1
)

# 5. Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)


# --- STEP 3: Start Training ---
# Only run fit if we haven't reached the max epochs yet
if initial_epoch < CONFIG["EPOCHS"]:
    print(f"🚀 Starting Training...")

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // CONFIG["BATCH_SIZE"],
        epochs=CONFIG["EPOCHS"],
        initial_epoch=initial_epoch,
        validation_data=val_generator,
        validation_steps=val_generator.samples // CONFIG["BATCH_SIZE"],
        # class_weight=train_class_weights,  <-- REMOVE THIS LINE. We used oversampling instead.
        callbacks=[early_stop, reduce_lr, checkpoint_best, checkpoint_last, csv_logger]
    )
    print("Training Complete.")

## Report Generation

In [ ]:
def generate_report(model, val_gen, output_path):
    print("Generating predictions (this may take a moment)...")

    # 1. Predictions
    # Reset generator to start so predictions match labels exactly
    val_gen.reset()

    # Calculate exact steps to ensure we cover all images (using math.ceil)
    steps = math.ceil(val_gen.samples / val_gen.batch_size)

    # Predict
    Y_pred = model.predict(val_gen, steps=steps, verbose=1)

    # Convert probabilities (e.g., [0.1, 0.8, 0.1]) to class ID (e.g., 1)
    y_pred = np.argmax(Y_pred, axis=1)

    # Get True labels (val_generator must have shuffle=False)
    y_true = val_gen.classes

    # Get Class Names
    labels = list(val_gen.class_indices.keys())

    # 2. Confusion Matrix Plot
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.ylabel('True Class')
    plt.xlabel('Predicted Class')

    cm_path = os.path.join(output_path, 'confusion_matrix.png')
    plt.savefig(cm_path)
    print(f"Confusion Matrix saved to {cm_path}")
    plt.show()

    # 3. Text Report (Precision, Recall, F1)
    report = classification_report(y_true, y_pred, target_names=labels)
    report_path = os.path.join(output_path, 'classification_report.txt')

    with open(report_path, "w") as f:
        f.write("FINAL PROJECT EVALUATION REPORT\n")
        f.write("===============================\n")
        f.write(f"Model: Custom CNN (From Scratch)\n")
        f.write(f"Image Size: {CONFIG['IMG_HEIGHT']}x{CONFIG['IMG_WIDTH']}\n")
        f.write("\nCLASSIFICATION METRICS:\n")
        f.write(report)

    print(f"Detailed Classification Report saved to {report_path}")
    print(report)

# --- RUN REPORT GENERATION ---
generate_report(model, val_generator, LOGS_DIR)

# --- PLOT HISTORY (ACCURACY & LOSS) ---
# This section now fills BOTH sides of the plot (Accuracy and Loss)
if 'history' in locals():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(15, 5))

    # Plot 1: Accuracy (Left Side)
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    # Plot 2: Loss (Right Side) - FIXED: This was missing in your code
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    # Save the combined plot
    plot_path = os.path.join(LOGS_DIR, 'training_history.png')
    plt.tight_layout()
    plt.savefig(plot_path)
    print(f"Training History Plot saved to {plot_path}")
    plt.show()
else:
    print("No training history found in memory. (Did you restart the kernel?)")